# AdminGPT: Your AI-Powered Administrative Assistant, powered by OpenAI's Assistant Framework  🚀
### Introduction
AdmiGPT is an AI-powered administrative assistant, harnessing the power of OpenAI's Assistant framework to seamlessly integrate with your email and calendar. Similar to Microsoft's Copilot, only better, it's designed to be your ultimate productivity partner, AdmiGPT offers an array of advanced features, making your administrative tasks simpler, faster, and more efficient.

AdminGPT is fully Open Source, so everything you need to run it for yourself is in this Github repo. This notebook helps you get started with AdminGPT, and walks you through how it's implemented. 

### Implementation
To begin, we're going to load our custom OpenAI Tools, which will interface with your email platform's API, and store any confidential and authentication information for the user in environmental variables.

In [1]:
from openai import OpenAI
import time, json, pprint, os
from tools.o365_toolkit import (
    o365search_emails,
    o365search_email,
    o365search_events,
    o365send_message,
    o365reply_message,
    o365send_event,
    o365find_free_time_slots,
    tools, toolkit_prompt
)
from datetime import datetime as dt
from tools.utils import authenticate

os.environ["OPENAI_API_KEY"] = "YOUR OPENAI KEY"
os.environ["CLIENT_ID"] = "YOUR CLIENT ID"
os.environ["CLIENT_SECRET"] = "YOUR CLIENT SECRET"

Next, we're going to set a few constants, which we will use throughout the code.

In [2]:
account = authenticate()
mailbox = account.mailbox()
mailboxsettings = mailbox.get_settings()
timezone = mailboxsettings.timezone
directory = account.directory(resource="me")
user = directory.get_current_user()
client_name = user.full_name
client_email = user.mail

# Set values for global variables
assistant_name = "Monica A. Ingenio"
current_date = dt.now()
formatted_date = current_date.strftime("%A, %B %d, %Y")
openai_api_key = os.environ.get("OPENAI_API_KEY")
debug = False
model = "gpt-4o-2024-08-06"
assistant_instructions = (
    "You are an AI Administrative Assistant called "
    + assistant_name
    + ", and I am your executive. My name is "
    + client_name
    + ". My email is "
    + client_email
    + ", and I am in the "
    + timezone
    + " timezone. You have access to my email and calendar. Today is "
    + formatted_date
    + ". ALWAYS use functions for determining free times and parsing proposed"
    + " meeting times in emails.\n"
    + toolkit_prompt
)

# Add the debug prompt if user runs with debug
if debug:
    debug_prompt = (
            "Please remember to track and document all interactions using the following format.\n "
            + "Start of Interaction: Briefly note the request. Follow these steps:\n"
            + "Prompt: Briefly describe the user request.\nTool Call: List the function used and key parameters.\n"
            + "Result: Summarize the result or action taken.\n"
            + "Repeat as needed for each step in the interaction. Conclude with any noteworthy observations.\n"
            + "End of Interaction\nIf I request a compilation of these interactions, ensure you're able to share"
            + " the documented interaction logs accurately and comprehensively, adhering to the detailed format I shared with you."
    )
else:
    debug_prompt = ""
assistant_instructions += debug_prompt

The next step is to create an OpenAI Assistant so we can interact with it, and a thread in which to run our prompts.

In [3]:
client = OpenAI(
    api_key=openai_api_key,
)

assistant = client.beta.assistants.create(
    name="AI Administrative Assistant",
    instructions=assistant_instructions,
    model=model,
    tools=tools,
)

thread = client.beta.threads.create()

To make execution easier in the future steps, we create a function to run prompts with only one call. We start by submitting a test prompt.

In [4]:
def run_prompt(prompt, client, assistant, thread):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    return run

prompt = 'Confirm you\'re ready by replying, "Hello, [MY FULL NAME]. How can I assist you today?"'

run = run_prompt(prompt, client, assistant, thread)

To make execution even easier in future steps, we create a function that polls the OpenAI API for a response to the prompt and executes tool calls, so we can use it to retrieve responses in the future.

In [5]:
def poll_for_response(client, thread, run, model, debug=False):
    LOOP_DELAY_SECONDS = 3

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status

        if status == "completed":
            response = client.beta.threads.messages.list(thread_id=thread.id)
            if response.data:
                return response.data[0].content[0].text.value
            break
        elif status == "requires_action":
            tools_outputs = []

            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                function_arguments = tool_call.function.arguments
                function_arguments = json.loads(function_arguments)

                # Case statement to execute each toolkit function
                if function_name == "o365search_emails":
                    output = o365search_emails(**function_arguments)
                elif function_name == "o365search_email":
                    output = o365search_email(**function_arguments)
                elif function_name == "o365search_events":
                    output = o365search_events(**function_arguments)
                elif function_name == "o365send_message":
                    output = o365send_message(**function_arguments)
                elif function_name == "o365send_event":
                    output = o365send_event(**function_arguments)
                elif function_name == "o365reply_message":
                    output = o365reply_message(**function_arguments)
                elif function_name == "o365find_free_time_slots":
                    output = o365find_free_time_slots(**function_arguments)

                # Clean the function output into JSON-like output
                output = pprint.pformat(output)
                tool_output = {"tool_call_id": tool_call_id, "output": output}
                tools_outputs.append(tool_output)

            if run.required_action.type == "submit_tool_outputs":
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tools_outputs
                )

        elif status == "failed":
            return "Run failed try again!"
            break

        if debug:
            print("The Assistant's Status is: " + status)

        time.sleep(LOOP_DELAY_SECONDS)

Now that we've submit our first prompt, we are going to poll for a response. If the response is, "How can I help you?", we know that the Assistant is working correctly, and we are ready to start using AdminGPT.

In [6]:
response = poll_for_response(client, thread, run, model, debug)
print(response)

Hello, Santiago Delgado. How can I assist you today?


To begin, we're going to perform the most simple task, which is summarizing an email from a specific sender.

In [7]:
prompt = (
    "Can you please concisely summarize the most recent email from Santiago Delgado"
    " including any proposed meeting times?"
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

The most recent email from Santiago Delgado is titled "Mike Portnoy coming back to Dream Theater?" Santiago is proposing a discussion on February 3, 2024, at 4:00 PM ET about Mike Portnoy rejoining Dream Theater.


Now, we're going to show AdminGPT's ability to interact with your calendar. We're going to check what events we have on the day that the email proposed.

In [8]:
prompt = (
    "Ok, thank you. What meetings do I have on February 3, 2024?"
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

On February 3, 2024, you have the following meetings scheduled:

1. Flight from Dallas to Cincinnati: 8:00 AM - 10:30 AM
2. Lunch Meeting with Management: 12:00 PM - 1:00 PM
3. Strategy Session: 3:00 PM - 5:00 PM
4. Flight from Cincinnati to Dallas: 6:00 PM - 8:30 PM


We now know when the email sender wants to meet, and what meetings I have on that day. So, let's see what times I have free that day.

In [9]:
prompt = "Ok, now what times am I free on February 3, 2024?"
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

On February 3, 2024, you are free during the following times:

1. 12:00 AM - 8:00 AM
2. 10:30 AM - 12:00 PM
3. 1:00 PM - 3:00 PM
4. 5:00 PM - 6:00 PM
5. 8:30 PM - Midnight


With all this information, we can now draft a response to the email letting the sender know that we can't meet at the proposed time, and propose other times to meet.

In [10]:
prompt = (
    "Please  draft a response to Santiago Delgado's latest email "
    "letting him know that I can't meet on the proposed times, and "
    "propose other free times on February 3,2024. Show me the draft."
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I've drafted the response to Santiago Delgado's email. Here's the draft:

---

Subject: Mike Portnoy coming back to Dream Theater?

Hi Santiago,

I appreciate the suggestion to discuss Mike Portnoy's return to Dream Theater on February 3, 2024, at 4:00 PM ET. Unfortunately, I am not available at that time.

However, I am free at the following times on February 3, 2024:
- 10:30 AM - 12:00 PM
- 1:00 PM - 3:00 PM
- 5:00 PM - 6:00 PM

Please let me know if any of these times work for you.

Best regards,

Santiago

---

The draft has been saved.


Sometimes, we will want the email to come from our AI administrative instead of the ourselves, so we can do that too.

In [11]:
prompt = (
        "Now, draft a response to Santiago Delgado's latest email "
        "letting him know that I can't meet on the proposed times, "
        "and propose other free times on February 3,2024. The email "
        "should come from you on behalf of me. Show me the draft."
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I've drafted the response on your behalf. Here's the draft:

---

Subject: Mike Portnoy coming back to Dream Theater?

Hi Santiago,

This is Monica, reaching out on behalf of Santiago Delgado. Santiago appreciates your suggestion to discuss Mike Portnoy's return to Dream Theater on February 3, 2024, at 4:00 PM ET. Unfortunately, he is not available at that time.

He is, however, free at the following times on February 3, 2024:
- 10:30 AM - 12:00 PM
- 1:00 PM - 3:00 PM
- 5:00 PM - 6:00 PM

Would any of these times work for you?

Thank you,

Monica A. Ingenio

---

The draft has been saved.


To conclude, I want to show that you can follow all these steps in one prompt.

In [13]:
prompt = (
    "Can you draft a response to Santiago Delgado's latest email. "
    "The email you draft should be from you as my assistant on behalf of me letting Santiago "
    "know if I can meet at the times he proposes, and if not, propose "
    "other times on the same day? Show me the draft."
)
run = run_prompt(prompt, client, assistant, thread)
response = poll_for_response(client, thread, run, model, debug)
print(response)

I've drafted a response on your behalf. Here's the draft:

---

Subject: Mike Portnoy coming back to Dream Theater?

Hi Santiago,

This is Monica, reaching out on behalf of Santiago Delgado. Santiago appreciates your suggestion to discuss Mike Portnoy's return to Dream Theater on February 3, 2024, at 4:00 PM ET. Unfortunately, he is not available at that time.

He is, however, free at the following times on February 3, 2024:
- 10:30 AM - 12:00 PM
- 1:00 PM - 3:00 PM
- 5:00 PM - 6:00 PM

Would any of these times work for you?

Thank you,

Monica A. Ingenio

---

The draft has been saved.
